In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import  pyarrow.parquet as pq

In [ ]:
dfItens = pd.read_csv("../../Microdados_enem_2019/DADOS/ITENS_PROVA_2019.csv", sep=';',encoding='iso-8859-1')

In [ ]:
dictSubjects = {"CN":[], "MT":[], "CH":[], "LC":[]}

In [ ]:
dictGabOrder = dictSubjects 


for subject in dictGabOrder.keys():
    dfCodes = pd.DataFrame()
    
    dfSubject = dfItens[dfItens["SG_AREA"]==subject]
    blueCode = dfSubject[dfSubject["TX_COR"]=="Azul"]["CO_PROVA"].unique()[0]

    if subject == "LC":

        dfCodes["CO_ITEM_0"] = dfSubject[(dfSubject["CO_PROVA"]==blueCode) & (dfSubject["TP_LINGUA"]!=1)][["CO_ITEM"]].reset_index(drop=True)
        dfCodes["CO_ITEM_1"] = dfSubject[(dfSubject["CO_PROVA"]==blueCode) & (dfSubject["TP_LINGUA"]!=0)][["CO_ITEM"]].reset_index(drop=True)

        dfUniqueProvaLingua = dfSubject[["CO_PROVA", "TP_LINGUA"]].drop_duplicates().dropna()

        for code, lingua in dfUniqueProvaLingua.itertuples(index=False):

            lingua = int(lingua)
            
            dfSubjectFiltered = dfSubject[(dfSubject["CO_PROVA"]==code) & ((dfSubject["TP_LINGUA"]==lingua) | (dfSubject["TP_LINGUA"].isna()))][["CO_ITEM", "CO_POSICAO"]]

            dfSubjectFiltered["CO_POSICAO"] = dfSubjectFiltered["CO_POSICAO"] - np.min(dfSubjectFiltered["CO_POSICAO"])
            dfSubjectFiltered.rename(columns= {"CO_ITEM":f'CO_ITEM_{lingua}'}, inplace=True) 
            
            dfCodes = dfCodes.merge(dfSubjectFiltered, on=f'CO_ITEM_{lingua}', how="left", suffixes=(None, f'_{code}_{lingua}'))
                    
        dfCodes.rename(columns= {"CO_POSICAO":f'CO_POSICAO_{blueCode}_{0}'}, inplace=True) 
        dictGabOrder[subject] = dfCodes
        
    
    else:
        dfCodes["CO_ITEM"] = dfSubject[dfSubject["CO_PROVA"]==blueCode][["CO_ITEM"]]
    
        for code in dfSubject['CO_PROVA'].unique():
            dfSubjectFiltered = dfSubject[dfSubject["CO_PROVA"]==code][["CO_ITEM", "CO_POSICAO"]]
            
            dfSubjectFiltered["CO_POSICAO"] = dfSubjectFiltered["CO_POSICAO"] - np.min(dfSubjectFiltered["CO_POSICAO"])
            
            dfCodes = dfCodes.merge(dfSubjectFiltered, on="CO_ITEM", how="left", suffixes=(None, f'_{code}'))
        
        dfCodes.rename(columns= {"CO_POSICAO":f'CO_POSICAO_{blueCode}'}, inplace=True) 
        dictGabOrder[subject] = dfCodes


In [267]:
dictResp = dictSubjects 

for subject in dictResp.keys():
    dictResp[subject] = pd.read_parquet(f'../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_BySubject/{subject}.parquet')

In [271]:
dictGabOrder["MT"].loc[0,"CO_POSICAO_522"] = 15
dictGabOrder["MT"]["CO_POSICAO_522"] = dictGabOrder["MT"]["CO_POSICAO_522"].astype('int32')


dictResp['CN'] = dictResp['CN'][dictResp['CN']['CO_PROVA_CN'] != 519].reset_index(drop=True)

In [ ]:
dictMasks = dictSubjects 

for subject in dictMasks.keys():
    dictMasks[subject] = pd.DataFrame()
    listOfValues = []

    dictColumnsToIterate = ['NU_INSCRICAO', f'CO_PROVA_{subject}']
   
    if subject == 'LC':
        dictColumnsToIterate.append('TP_LINGUA')
 
    for index, row in dictResp[subject][dictColumnsToIterate].iterrows():
        codigo = int(row[f'CO_PROVA_{subject}'])

        co_posicao = f'CO_POSICAO_{codigo}_{int(row["TP_LINGUA"])}' if subject == 'LC' else f'CO_POSICAO_{codigo}'

        dictPos = np.dict(dictGabOrder[subject][co_posicao]) + 1 
        dictPos = np.insert(dictPos, 0 , 0)
        dictPos = np.insert(dictPos, 46 , 46)
        dictPos = np.insert(dictPos, 47 , 47)
    
        if subject == 'LC':
           dictPos = np.insert(dictPos, 48 , 48)

        listOfValues.append(dictPos)

    dictMasks[subject] = listOfValues

In [ ]:
dictRespOrdered = dictSubjects 

for subject in dictRespOrdered.keys():
    columns = dictResp[subject].columns
    
    dfSubject = pd.concat([ pd.DataFrame(  [row[dictMasks[subject][index]].values], columns=columns )  for index, row in dictResp[subject].iterrows()], ignore_index=True)

    dictRespOrdered[subject] = dfSubject

In [ ]:
for subject, dfSubject in dictRespOrdered.items():
    
    table = pa.Table.from_pandas(dfSubject.reset_index(drop=True))

    
    # Definir onde será salvo o novo dataset filtrado por área
    path = f'../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_BySubjectOrdered/{subject}.parquet'

    # Gerar um parquet da tabela
    pq.write_to_dataset(
        table,
        root_path=path,
    )